In [1]:
# trying this again after adding index for year

# play with some sql selections to make sure things are working

# todo clean up these imports

from pytorch_pretrained_bert import BertModel, BertTokenizer
import torch

import time
import mysql.connector as mysql

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import Row, StructType, StructField, IntegerType, StringType

import os
import time
import random
#import mysql.connector as mysql   # import gc




import csv

#import igraph

import time
import numpy as np

import matplotlib.pyplot as plt
#import cairocffi as cairo

import seaborn as sns
import umap
import statsmodels.api as sm  # for kdemultivariate 

import json


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
SUBMIT_ARGS = "--driver-class-path file:///home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar --jars file:///home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS

db_name = 'test_pubmed'  # db name collisons? https://stackoverflow.com/questions/14011968/user-cant-access-a-database
url = "jdbc:mysql://localhost:3306/{}?useUnicode=true&useJDBCCompliantTimezoneShift=true&useLegacyDatetimeCode=false&serverTimezone=America/Chicago".format(db_name)  # mysql runs on port 3306
client_config = {'unix_socket':'/home/brendanchambers/.sql.sock',
                'database': db_name}  # for python connector

In [3]:
print('initializing spark')
# init spark
conf = SparkConf()
conf = (conf.setMaster('local[*]')
       .set('spark.driver.memory','28G')
       .set("spark.jars", "/home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar"))        
'''
.set('spark.executor.memory','1G')  # 20
.set('spark.driver.memory','1G')   # 40
.set('spark.driver.maxResultSize','500M')  #.set('spark.storage.memoryFraction',0))  # this setting is now a legacy option
.set('spark.python.worker.reuse', 'false')
.set('spark.python.worker.memory','512m')
.set('spark.executor.cores','1'))
'''
sc = SparkContext(conf=conf)
#sc.addJar('home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar')  # temp
spark = SparkSession(sc)  # don't need this for vanilla RDDs

print(sc._conf.getAll())

initializing spark
[('spark.repl.local.jars', 'file:///home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar'), ('spark.jars', '/home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar'), ('spark.driver.host', 'midway2-0058.rcc.local'), ('spark.executor.id', 'driver'), ('spark.app.name', 'pyspark-shell'), ('spark.app.id', 'local-1565063711164'), ('spark.driver.port', '45801'), ('spark.rdd.compress', 'True'), ('spark.driver.extraClassPath', 'file:///home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.driver.memory', '28G'), ('spark.submit.deployMode', 'client'), ('spark.ui.showConsoleProgress', 'true')]


In [5]:
# test read

tablename_ = 'metadata'

sql = "(SELECT * FROM {}) AS t".format(tablename_)
print(sql)

start_time = time.time()
df_meta = spark.read.format('jdbc').option("url", url)\
                              .option("dbtable", sql)\
                              .load().repartition(9).cache()
end_time = time.time()
print("dataframe loaded in {} s".format(end_time - start_time))

print(df_meta.rdd.getNumPartitions())

(SELECT * FROM metadata) AS t
dataframe loaded in 3.0904016494750977 s
9


In [9]:
# try the same query with python mysql connector

# RESULTS this is slower than the pyspark dataframe method
#   suggesting that the indexed columns are being used smartly by pyspark
#   while also utilizing parallelization

# ~56 s for 2007
# ~62 s for 2017


year_ = 2017
sql = 'SELECT pmid FROM metadata WHERE year = {};'.format(year_)

db = mysql.connect(**client_config)
cursor = db.cursor()
start_time = time.time()
cursor.execute(sql)
print('elapsed time: {}'.format(time.time() - start_time))

start_time = time.time()
published_pmids = cursor.fetchall()

print('elapsed time: {}'.format(time.time() - start_time))

cursor.close()
db.close()

elapsed time: 0.07869338989257812
(24275634, 'Multimodal Sensory Distortions in Postpartum Exacerbation of Schizophrenia.', 2017)

(30225395, None, 2017)

(30225396, 'The Challenging Conundrum of Diagnosing and Managing', 2017)

(30225397, 'Practical Strategies for Assessing Patient Physical Activity Levels in Primary Care.', 2017)

(30225398, 'Nonexercise Estimated Cardiorespiratory Fitness and Mortality Due to All Causes and Cardiovascular Disease: The NHANES III Study.', 2017)

(30225399, 'National Trends in the Incidence, Management, and Outcomes of Heart Failure Complications in Patients Hospitalized for ST-Segment Elevation Myocardial Infarction.', 2017)

(30225400, 'Acute Alcoholic Hepatitis: Natural History and Predictors of Mortality Using a Multicenter Prospective Study.', 2017)

(30225401, 'Experience and Outcomes at a Specialized', 2017)

(30225402, 'Normalization of Testosterone Levels After Testosterone Replacement Therapy Is Not Associated With Reduced Myocardial Infarct

True

In [19]:
# get cited papers


# use jdbc to execute sql join and then count using pyspark dataframe

tablename_ = 'citations'

year_ = 1959
sql = 'SELECT pmid FROM metadata WHERE year = {};'.format(year_)  # filter using JOIN todo
print(sql)

start_time = time.time()
df = spark.read.format('jdbc').option("url", url)\
                              .option("dbtable", sql)\
                              .load().repartition(9).cache()
end_time = time.time()
print("dataframe loaded in {} s".format(end_time - start_time))









SELECT pmid FROM metadata WHERE year = 1959;


Py4JJavaError: An error occurred while calling o156.load.
: java.sql.SQLSyntaxErrorException: You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'SELECT pmid FROM metadata WHERE year = 1959; WHERE 1=0' at line 1
	at com.mysql.cj.jdbc.exceptions.SQLError.createSQLException(SQLError.java:120)
	at com.mysql.cj.jdbc.exceptions.SQLError.createSQLException(SQLError.java:97)
	at com.mysql.cj.jdbc.exceptions.SQLExceptionsMapping.translateException(SQLExceptionsMapping.java:122)
	at com.mysql.cj.jdbc.ClientPreparedStatement.executeInternal(ClientPreparedStatement.java:955)
	at com.mysql.cj.jdbc.ClientPreparedStatement.executeQuery(ClientPreparedStatement.java:1005)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.resolveTable(JDBCRDD.scala:60)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRelation.<init>(JDBCRelation.scala:115)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelation

In [52]:
# sample query (ok to delete)




SELECT pmid FROM metadata WHERE year = 1959;
elapsed time: 0.7483458518981934
[24545123, 24545124, 24545125, 24545126, 24545127, 24545140, 24545141, 24545142, 24545143, 24545144]


In [49]:
# sample join

# ~1 s for 1959
# ~20 min for 2017

db = mysql.connect(**client_config)
year_ = 1959
sql = '''SELECT referenced
            FROM citations 
                LEFT JOIN metadata
                ON citations.citing = metadata.pmid
            WHERE year = {};'''.format(year_)  # filter using JOIN todo

print(sql)
cursor = db.cursor()
start_time = time.time()
cursor.execute(sql)
year_citations = [x for [x] in cursor.fetchall()]
print(year_citations[:10])


end_time = time.time()
print('elapsed time: {}'.format(end_time - start_time))

SELECT referenced
            FROM citations 
                LEFT JOIN metadata
                ON citations.citing = metadata.pmid
            WHERE year = 1959;
[12997665, 13149802, 13568043, 13568047, 13588436, 13621645, 21009155, 15402898, 12999049, 14910862]
elapsed time: 1.9896306991577148


In [54]:
save_data = {'publications': year_publications,
            'citations': year_citations}

with open('json/pubmed_state_{}'.format(year_), 'w') as f:
    json.dump(save_data, f)

In [4]:
#  loop through the years and produce published pmids and cited pmids
#    since I'm hurrying, just grabbing the code from above and copying

for year_ in range(2004, 2020): #  range(1958, 2018):
    start_time = time.time()
    
    # get published article pmids from this year
    sql = 'SELECT pmid FROM metadata WHERE year = {};'.format(year_)  # filter using JOIN todo
    print(sql)
    
    db = mysql.connect(**client_config)
    cursor = db.cursor()
    cursor.execute(sql)
    year_publications = [x for [x] in cursor.fetchall()]

    cursor.close()
    db.close()
    
    # get cited article pmids from this year
    sql = '''SELECT referenced
            FROM citations 
                LEFT JOIN metadata
                ON citations.citing = metadata.pmid
            WHERE year = {};'''.format(year_)  # filter using JOIN todo

    print(sql)
    db = mysql.connect(**client_config)
    cursor = db.cursor()
    cursor.execute(sql)
    year_citations = [x for [x] in cursor.fetchall()]
    
    cursor.close()
    db.close()
    
    # write to json file
    save_data = {'publications': year_publications,
            'citations': year_citations}
    with open('json/pubmed_state_{}'.format(year_), 'w') as f:
        json.dump(save_data, f)

    end_time = time.time()
    print('elapsed time: {}'.format(end_time - start_time))



SELECT pmid FROM metadata WHERE year = 2004;
SELECT referenced
            FROM citations 
                LEFT JOIN metadata
                ON citations.citing = metadata.pmid
            WHERE year = 2004;
elapsed time: 435.9970154762268
SELECT pmid FROM metadata WHERE year = 2005;
SELECT referenced
            FROM citations 
                LEFT JOIN metadata
                ON citations.citing = metadata.pmid
            WHERE year = 2005;
elapsed time: 397.86027431488037
SELECT pmid FROM metadata WHERE year = 2006;
SELECT referenced
            FROM citations 
                LEFT JOIN metadata
                ON citations.citing = metadata.pmid
            WHERE year = 2006;
elapsed time: 443.1734552383423
SELECT pmid FROM metadata WHERE year = 2007;
SELECT referenced
            FROM citations 
                LEFT JOIN metadata
                ON citations.citing = metadata.pmid
            WHERE year = 2007;
elapsed time: 439.329074382782
SELECT pmid FROM metadata WHERE year

In [ ]:
sc.stop()